###Код

In [145]:
from pprint import pprint
from lxml import html
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
import json
import re

HEADHUNTER

In [146]:
def hh_parsing():
  vacancy_dict = {1: 'prodavec', 2: 'menedzher', 3: 'uborshik'} 
  vacancy = int(input('''Выберите профессию для поиск в HeadHunter: 
1 - продавец
2 - менеджер
3 - уборщик

'''))
  pages_ammount = int(input('Введите количество страниц с которых хотите получить информацию, не более 40: '))
  n = 0
  global hh_list
  hh_list = []
  c = []
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
  while n < pages_ammount:
    url = f'https://hh.ru/vacancies/{vacancy_dict[vacancy]}?page={n}'
    response = requests.get(url=url, headers=headers)
    dom = bs(response.text, 'html.parser')
    tag_hh = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for t in tag_hh:  
      tag_vacancy_name = t.find('a', {'class':'serp-item__title'}).text
      tag_hh_link = t.find('a', {'class': 'serp-item__title'})['href']
      tag_vacancy_salary = t.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})
      try:
        tag_vacancy_salary = tag_vacancy_salary.text.strip()
      except AttributeError:
        tag_vacancy_salary = 'з/п не указана'
      vacancies_dict = {
      'source': 'HH',
      'name': tag_vacancy_name,
      'salary': tag_vacancy_salary.replace('\u202f', ""),
      'link': tag_hh_link
  }
      hh_list.append(vacancies_dict)
    # time.sleep(2)
    n += 1
  # return hh_list

SUPERJOB

In [147]:
def sj_parsing():
  sj_vacancy_dict = {1: 'Продавец', 2: 'Менеджер', 3: 'Уборщик'} 
  sj_vacancy = int(input('''\nВыберите профессию для поиска в SuperJob: 
1 - продавец
2 - менеджер
3 - уборщик

'''))
  pages_ammount = int(input('Введите количество страниц с которых хотите получить информацию, не более 5: '))
  n = 0
  global sj_list
  sj_list = []
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
  while n < pages_ammount:
    url = f'https://www.superjob.ru/vacancy/search/?keywords={sj_vacancy_dict[sj_vacancy]}?page={n}]'
    response = requests.get(url=url, headers=headers)
    dom = bs(response.text, 'html.parser')
    tag_sj = dom.find_all('div', {'class': '_1iE-a _36PRW'})
    for t_sj in tag_sj:
      tag_sj_link = 'https://www.superjob.ru' + t_sj.find('a', {'class': '_1IHWd'})['href']
      tag_vacancy_sj_salary = t_sj.find('span', {'class':'_2eYAG rIDaO oDIMq _33qju _3T7lp'}).text.strip()
      # tag_vacancy_sj_salary_final = re.sub('до|от|₽', "", tag_vacancy_sj_salary)
      tag_vacancy_sj_name = t_sj.find('a', {'class':'_1IHWd'})
      try:
        tag_vacancy_sj_name = tag_vacancy_sj_name.text.strip()
      except AttributeError:
        tag_vacancy_sj_name = 'должность не указана'
      vacancies_sj_dict = {
      'source': 'SJ',
      'name': tag_vacancy_sj_name,
      'salary': tag_vacancy_sj_salary.replace('\xa0', ""),
      'link': tag_sj_link
  }
      sj_list.append(vacancies_sj_dict)
    # time.sleep(2)
    n += 1
  # return hh_list

In [148]:
def job_search():
  hh_parsing()
  sj_parsing()
  global final_list
  final_list = hh_list + sj_list
  return pd.DataFrame(final_list)

###ИТОГ

In [149]:
result = job_search()
result

Выберите профессию для поиск в HeadHunter: 
1 - продавец
2 - менеджер
3 - уборщик

2
Введите количество страниц с которых хотите получить информацию, не более 40: 1

Выберите профессию для поиска в SuperJob: 
1 - продавец
2 - менеджер
3 - уборщик

3
Введите количество страниц с которых хотите получить информацию, не более 5: 2


,source,name,salary,link
0,HH,Менеджер,50000 – 70000 руб.,https://adsrv.hh.ru/click?b=368311&c=45&meta=g...
1,HH,Менеджер по продажам (стажер),60000 – 100000 руб.,https://hh.ru/vacancy/69017868?query=%D0%BC%D0...
2,HH,Менеджер оптовых продаж,70000 – 230000 руб.,https://hh.ru/vacancy/77262869?query=%D0%BC%D0...
3,HH,Менеджер по продажам,60000 – 250000 руб.,https://hh.ru/vacancy/77198960?query=%D0%BC%D0...
4,HH,Удаленный оператор / помощник менеджера продаж...,до 140000 руб.,https://hh.ru/vacancy/77194747?query=%D0%BC%D0...
...,...,...,...,...
126,SJ,Уборщица офисных помещений,до30000₽,https://www.superjob.ru/vakansii/uborschica-of...
127,SJ,"Клинер, уборщик",от75000₽,https://www.superjob.ru/vakansii/kliner-456339...
128,SJ,Уборщица,53000₽,https://www.superjob.ru/vakansii/uborschica-45...
129,SJ,Уборщик служебных помещений,до30000₽,https://www.superjob.ru/vakansii/uborschik-slu...


In [150]:
with open('/content/drive/MyDrive/ОБУЧЕНИЕ В GB. МАТЕРИАЛЫ/Модуль 6/data.json', 'w') as outfile:
  json.dump(final_list, outfile)

In [151]:
!apt install mongodb > log

In [152]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [153]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [154]:
from pymongo import MongoClient

In [155]:
client = MongoClient()

In [156]:
db = client.vacancies

In [157]:
collection_vacancies_hh_sj = db.vacancies_hh_sj

In [158]:
with open('/content/drive/MyDrive/ОБУЧЕНИЕ В GB. МАТЕРИАЛЫ/Модуль 6/data.json') as json_file:
    data = json.load(json_file)

In [173]:
for d in data:
  db.collection_vacancies_hh_sj.insert_one(d)

In [ ]:
for d in data:
  if not len(list(db.collection_vacancies_hh_sj.find({'link': d['link']}))):
    db.collection_vacancies_hh_sj.insert_one(d)

In [177]:
print(len(list(db.collection_vacancies_hh_sj.find())))

131


In [ ]:
for d in db.collection_vacancies_hh_sj.find({'source': 'HH'}):
  print(d)

In [186]:
for d in db.collection_vacancies_hh_sj.find({'name': 'Уборщик'}):
  print(d)

{'_id': ObjectId('63f50fdd0790bf834e80cb08'), 'source': 'SJ', 'name': 'Уборщик', 'salary': 'от45000₽', 'link': 'https://www.superjob.ru/vakansii/uborschik-43119980.html'}
{'_id': ObjectId('63f50fdd0790bf834e80cb1c'), 'source': 'SJ', 'name': 'Уборщик', 'salary': 'от35000₽', 'link': 'https://www.superjob.ru/vakansii/uborschik-45815240.html'}
{'_id': ObjectId('63f50fdd0790bf834e80cb30'), 'source': 'SJ', 'name': 'Уборщик', 'salary': 'от45000₽', 'link': 'https://www.superjob.ru/vakansii/uborschik-43119980.html'}
{'_id': ObjectId('63f50fdd0790bf834e80cb44'), 'source': 'SJ', 'name': 'Уборщик', 'salary': 'от35000₽', 'link': 'https://www.superjob.ru/vakansii/uborschik-45815240.html'}
